In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
#converting categorical to numerical value
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Smoothing parameter
    'norm': [True, False]  # Whether to normalize the weights
}

# Set up the GridSearchCV object
grid = GridSearchCV(ComplementNB(), param_grid, 
                    refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fitting the model for grid search
grid.fit(X_train, y_train)

# To see the best parameters and the best estimator
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'alpha': 0.1, 'norm': False}
Best Estimator: ComplementNB(alpha=0.1)


In [13]:
# Store cross-validation results
cv_results = grid.cv_results_
# Uncomment to print all cross-validation results
# print(cv_results)


# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)

# Using the final best fit model to test X_test
grid_predictions = grid.predict(X_test)

In [14]:
# Compute and print the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions )  

In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.67      0.47      0.55        85
           1       0.39      0.59      0.47        49

    accuracy                           0.51       134
   macro avg       0.53      0.53      0.51       134
weighted avg       0.57      0.51      0.52       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.5834333733493396)

In [18]:
print(cm)

[[40 45]
 [20 29]]


In [19]:
grid.predict([[40,300,1,]])

C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but ComplementNB was fitted with feature names
  warnings.warn(


array([0])

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.016625,0.002947,0.472251,0.416930,0.1,True,"{'alpha': 0.1, 'norm': True}",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,6
1,0.015994,0.005048,0.432055,0.288650,0.1,False,"{'alpha': 0.1, 'norm': False}",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
2,0.024595,0.023240,0.024537,0.010315,0.5,True,"{'alpha': 0.5, 'norm': True}",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,6
3,0.014062,0.001842,0.014927,0.002355,0.5,False,"{'alpha': 0.5, 'norm': False}",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
4,0.011686,0.002815,0.013988,0.006561,1.0,True,"{'alpha': 1.0, 'norm': True}",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,6
5,0.008765,0.003157,0.011692,0.003415,1.0,False,"{'alpha': 1.0, 'norm': False}",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
6,0.013688,0.003239,0.015507,0.003870,1.5,True,"{'alpha': 1.5, 'norm': True}",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,6
7,0.009631,0.002539,0.015991,0.008636,1.5,False,"{'alpha': 1.5, 'norm': False}",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
8,0.011178,0.003459,0.013054,0.005082,2.0,True,"{'alpha': 2.0, 'norm': True}",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,6
9,0.015643,0.004311,0.015258,0.002342,2.0,False,"{'alpha': 2.0, 'norm': False}",0.509839,0.614012,0.547170,0.482156,0.556133,0.541862,0.044783,5
